<h1>Primo Progetto<h1>

In [1]:
# Importo le librerie richieste
import pandas as pd
from serpapi import GoogleScholarSearch
import json

In [2]:
GoogleScholarSearch.SERP_API_KEY = "2ffb3e2b7c2820b111bb5259892890ee2691e1afe058a01fe13f62e6ceecbb8d"

#### 1 | Utilizzo della libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati nel file nodes.csv

In [3]:
# Importo il file nodes.csv
df = pd.read_csv('data/nodes.csv')
display(df)

,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,Computer Science
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,"Computer Science, Crowdsourcing"
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,690,"Artificial Intelligence, Crowdsourcing and Hum..."
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4739,"Information retrieval, mobile systems, scholar..."
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2483,"Information Retrieval, Text Analytics, Data Sc..."
5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5348,"Information Retrieval, Semantic Web, Human Com..."
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."


<h1>NON RUNNARE FINO A NUOVO ORDINE, SERVE SOLO PER SCARICARE I DATI<h1>

```python

In [ ]:

updated_df = pd.DataFrame(columns=['author_id', 'cited_by', 'interests'])

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_name = row['name']
    author_affliation = row['affiliations']
        
    params = {
        "engine" : "google_scholar_profiles",
        "hl": "en",
        "mauthors":f'{author_name},{author_affliation}'
    }
    search = GoogleScholarSearch(params)
    results = search.get_dict()['profiles']
    
    for profile in results:
        author_id = profile['author_id']
        cited_by = profile['cited_by']
        
        interests = []
        
        for interest in profile.get('interests', ):
            title = interest.get('title', '')
            interests.append(title)
    
    interests_str = ', '.join(interests)
    
    # Aggiungo una nuova riga al dataframe updated_df
    new_row = {'author_id': author_id, 'cited_by': cited_by, 'interests': interests_str}
    updated_df = pd.concat([updated_df, pd.DataFrame([new_row])], ignore_index=True)

df = pd.merge(df, updated_df, left_index=True, right_index=True)
display(df)



In [ ]:
# Sovrascrivo il file nodes.csv con le nuove informazioni trovate
df.to_csv('data/nodes.csv', index=False)

#### 2 | Per ogni autore utilizzo il suo ID per accedere al relativo profilo GoogleScholar e scaricare l’elenco dei suoi coautori, sempre via SerpAPI

In [12]:
df = pd.read_csv('data/nodes.csv')

# Creo le liste vuote per memorizzare i nomi dei co-autori e successivamente le loro informazioni
co_authors = []
info_co_authors = []

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_id = row['author_id']
    author_name = row['name']    
        
    params = {
        "engine" : "google_scholar_author",
        "hl": "en",
        "author_id": author_id  
    }
    
    search = GoogleScholarSearch(params)
    results = search.get_dict()['co_authors']
    #print(results)
    
    # Estraggo i nomi dei co-autori per ogni autore originario, catturo il legame che c'è tra ogni autore con i suoi co-autori
    # Sto quindi catturando gli archi del mio grafo
    for co_author in results:
        co_authors.append(
            {
                "author1" : author_name,
                "author2" : co_author['name']
            }
        )
        
    # Estraggo le informazioni relative ai co-autori che sono in grado di estrarre già tramite questa ricerca SerpAPI
    for co_author in results:
        info_co_authors.append(
            {
                "name" : co_author['name'],
                "affiliations": co_author.get('affiliations'),
                "author_id" : co_author['author_id']
            }
        )
    
    
coauthors_df = pd.DataFrame(columns=['name', 'affiliations', 'author_id'])
coauthorship_df = pd.DataFrame(columns=['author1','author2'])

coauthors_df = pd.concat([coauthors_df, pd.DataFrame(info_co_authors)], ignore_index=True)
coauthorship_df = pd.concat([coauthorship_df, pd.DataFrame(co_authors)], ignore_index=True)

# il dataframe coauthors_df corrisponde al dataframe contenente le informazioni, ancora non complete, sui nodi del grafo 
display(coauthors_df)
# il dataframe coauthorship_df corrisponde al dataframe contenente le informazioni, complete, sugli archi del grafo 
display(coauthorship_df)




,name,affiliations,author_id
0,Kevin Roitero,University of Udine,1xd52jMAAAAJ
1,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ
2,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ
3,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ
4,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ
...,...,...,...
122,Pavlos Vougiouklis,Huawei Technologies,9J7YeR0AAAAJ
123,Jonathon Hare,Professor of Machine Learning and Computer Vis...,UFeON5oAAAAJ
124,Carlo Alberto Beltrami,"Professore, Università di Udine",EZrMWWMAAAAJ
125,Richard Gomer,University of Southampton,5ZTLglQAAAAJ


,author1,author2
0,David La Barbera,Kevin Roitero
1,David La Barbera,Stefano Mizzaro
2,David La Barbera,Gianluca Demartini
3,David La Barbera,Damiano Spina
4,David La Barbera,Michael Soprano
...,...,...
122,Eddy Maddalena,Pavlos Vougiouklis
123,Eddy Maddalena,Jonathon Hare
124,Eddy Maddalena,Carlo Alberto Beltrami
125,Eddy Maddalena,Richard Gomer


In [13]:

# Per trovare 'cited_by' e 'interests' dei co-autori devo fare un'ulteriore ricerca sull base dei loro nomi e dopodichè posso aggiungere queste info al dataframe
# Itero sulla lista co_authors che contiene tutti i nomi dei co_authori, per ognuno di loro effettuo una ricerca tramite nome
info_df = pd.DataFrame(columns=['cited_by', 'interests'])


for index, co_author in coauthors_df.iterrows():
    co_author_name = row['name']
    co_author_affiliation = row['affiliations']
    
    
    params = {
        "engine" : "google_scholar_profiles",
        "hl": "en",
        "mauthors":f'{co_author_name},{co_author_affiliation}'
    }
    
    search = GoogleScholarSearch(params)
    results = search.get_dict()['profiles']
    
    for profile in results:
        cited_by = profile['cited_by']
        
        interests = []
        
        for interest in profile.get('interests', ):
            title = interest.get('title', '')
            interests.append(title)
    
    interests_str = ', '.join(interests)
    
    # Add the information to the coauthors_df dataframe
    new_row = {'cited_by': cited_by, 'interests': interests_str}
    info_df = pd.concat([info_df, pd.DataFrame([new_row])], ignore_index=True)
    
coauthors_df = pd.merge(coauthors_df, info_df, left_index=True, right_index=True)    
display(coauthors_df)

,name,affiliations,author_id,cited_by,interests
0,Kevin Roitero,University of Udine,1xd52jMAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
1,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
2,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
3,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
4,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
...,...,...,...,...,...
122,Pavlos Vougiouklis,Huawei Technologies,9J7YeR0AAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
123,Jonathon Hare,Professor of Machine Learning and Computer Vis...,UFeON5oAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
124,Carlo Alberto Beltrami,"Professore, Università di Udine",EZrMWWMAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
125,Richard Gomer,University of Southampton,5ZTLglQAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."


In [19]:
nodes = pd.concat([df, coauthors_df], ignore_index=True).drop_duplicates()
display(nodes)
nodes.to_csv('data/nodes.csv', index=False)

# TODO: sono da rimuovere le coppie simmetriche????
edges = coauthorship_df
edges.to_csv('data/edges.csv', index=False)


,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,Computer Science
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,"Computer Science, Crowdsourcing"
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,690,"Artificial Intelligence, Crowdsourcing and Hum..."
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4739,"Information retrieval, mobile systems, scholar..."
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2483,"Information Retrieval, Text Analytics, Data Sc..."
...,...,...,...,...,...
128,Dante Degl'Innocenti,Datamantix,Ay6L7NsAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
129,Pavlos Vougiouklis,Huawei Technologies,9J7YeR0AAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
130,Jonathon Hare,Professor of Machine Learning and Computer Vis...,UFeON5oAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."
131,Carlo Alberto Beltrami,"Professore, Università di Udine",EZrMWWMAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."


<h1>RUNNIAMO DA QUA IN GIU<h1>

In [ ]:
nodes = pd.read_csv('data/nodes.csv')
edges = pd.read_csv('data/edges.csv')